In [4]:
# File: portfolio_manager.py
import os
import json
import yfinance as yf
import pandas as pd
from typing import TypedDict
from fredapi import Fred
from sec_edgar_downloader import Downloader
from langgraph.graph import StateGraph, END
from langchain_openai import AzureChatOpenAI
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

parser = JsonOutputParser()


# ----------------- Configuration -----------------
FRED_API_KEY = os.getenv("FRED_API_KEY")
# AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
NEO4J_URI = "bolt://localhost:7687"
NEO4J_AUTH = ("neo4j", "password")

from dotenv import load_dotenv
import os
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


# Predefined portfolio constituents
PORTFOLIO_SYMBOLS = [
    'AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA',
    'JNJ', 'PG', 'V', 'MA', 'NVDA',          # 10 stocks
    'GLD',                                   # Gold ETF
    'TLT', 'BND'                             # Bond ETFs
]

SYMBOL_SECTORS = {
    # Stocks
    'AAPL': 'Technology',
    'MSFT': 'Technology',
    'GOOG': 'Technology',
    'AMZN': 'Consumer Discretionary',
    'TSLA': 'Consumer Discretionary',
    'JNJ': 'Healthcare',
    'PG': 'Consumer Staples',
    'V': 'Financials',
    'MA': 'Financials',
    'NVDA': 'Technology',
    
    # Alternative assets
    'GLD': 'Commodity',
    'TLT': 'Bonds',
    'BND': 'Bonds'
}

GOLD_SYMBOL = 'GLD'
BOND_SYMBOLS = ['TLT', 'BND']

# Initialize services
fred = Fred(api_key=FRED_API_KEY)

llm = ChatGroq(
    model="deepseek-r1-distill-qwen-32b",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

c:\Users\Subrata Samanta\miniconda3\envs\genai\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
query = """Optimize portfolio for medium risk tolerance with:
- Maximum 25% tech sector exposure
- Minimum 20% allocation to safe-haven assets
- 5-year investment horizon"""


In [6]:
graph = Neo4jGraph(
            url=os.getenv("NEO4J_URI"),
            username=os.getenv("NEO4J_USERNAME"),
            password=os.getenv("NEO4J_PASSWORD"),
            database="neo4j")

In [7]:
graph.query("RETURN 1 AS test")

[{'test': 1}]

In [8]:
def extract_requirements(user_query):
    # Safely get query from state
    # user_query = state.get("user_query", "")
    
    template = """Analyze portfolio request and extract:
    {query}

    Return JSON with:
    - risk_tolerance: low/medium/high
    - time_horizon: years
    - constraints: {{
        max_sector_allocation: {{sector: max_percent}},
        min_alternative_allocation: percentage
    }}"""

    prompt = PromptTemplate(template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()})
    

    chain = prompt | llm | parser
    response = chain.invoke({"query": user_query})

    print("output from extract_requirements:", response)
    
    # response = llm.invoke(prompt)
    return response

In [9]:
requirements= extract_requirements(query)

output from extract_requirements: {'risk_tolerance': 'medium', 'time_horizon': 5, 'constraints': {'max_sector_allocation': {'tech': 25}, 'min_alternative_allocation': 20}}


In [10]:
requirements

{'risk_tolerance': 'medium',
 'time_horizon': 5,
 'constraints': {'max_sector_allocation': {'tech': 25},
  'min_alternative_allocation': 20}}

In [11]:

def get_market_data(PORTFOLIO_SYMBOLS,period: str = "3y"):
    """Fetch historical prices for all portfolio assets"""
    return yf.download(PORTFOLIO_SYMBOLS, period=period,auto_adjust=False)['Adj Close']

def get_economic_data():
    """Fetch key economic indicators from FRED"""
    return {
        'DGS10': fred.get_series('DGS10'),   # 10-Year Treasury Rate
        'CPI': fred.get_series('CPIAUCSL'),  # Consumer Price Index
        'UNRATE': fred.get_series('UNRATE'), # Unemployment Rate
        'GFDEBTN': fred.get_series('GFDEBTN') # Federal Debt
    }


In [12]:
market_data=get_market_data(PORTFOLIO_SYMBOLS,period="3y")

[*********************100%***********************]  13 of 13 completed


In [18]:
market_data

Ticker,AAPL,AMZN,BND,GLD,GOOG,JNJ,MA,MSFT,NVDA,PG,TLT,TSLA,V
Date,,,,,,,,,,,,,
2022-04-01,171.530609,163.559998,72.140381,179.500000,140.036713,163.086349,357.529327,301.616974,26.669739,143.901474,119.766136,361.529999,221.187485
2022-04-04,175.594772,168.346497,72.195038,180.369995,142.965347,161.512131,360.417297,307.026947,27.316715,142.964340,118.933807,381.816681,222.535950
2022-04-05,172.268631,164.054993,71.466072,179.240005,140.398026,162.555511,356.890808,303.040070,25.889975,143.465378,116.246796,363.753326,220.923645
2022-04-06,169.090134,158.755997,71.247368,179.660004,136.529327,166.783936,347.293762,291.947021,24.368385,145.515961,115.305916,348.586670,214.005432
2022-04-07,169.395203,157.784500,71.101578,180.339996,135.821671,166.353745,343.841003,293.769897,24.169701,147.399506,114.437386,352.420013,211.210800
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-25,223.750000,205.710007,73.089996,278.470001,172.789993,161.020004,545.159973,395.160004,120.690002,162.850006,89.760002,288.140015,344.619995
2025-03-26,221.529999,201.130005,72.930000,278.239990,167.139999,161.720001,549.070007,389.970001,113.760002,166.580002,89.169998,272.059998,344.179993
2025-03-27,223.850006,201.360001,72.900002,281.970001,164.080002,163.130005,557.570007,390.579987,111.430000,168.710007,88.910004,273.130005,349.859985


In [13]:
economic_data=get_economic_data()

In [16]:
economic_data

{'DGS10': 1962-01-02    4.06
 1962-01-03    4.03
 1962-01-04    3.99
 1962-01-05    4.02
 1962-01-08    4.03
               ... 
 2025-03-24    4.34
 2025-03-25    4.31
 2025-03-26    4.35
 2025-03-27    4.38
 2025-03-28    4.27
 Length: 16499, dtype: float64,
 'CPI': 1947-01-01     21.480
 1947-02-01     21.620
 1947-03-01     22.000
 1947-04-01     22.000
 1947-05-01     21.950
                ...   
 2024-10-01    315.564
 2024-11-01    316.449
 2024-12-01    317.603
 2025-01-01    319.086
 2025-02-01    319.775
 Length: 938, dtype: float64,
 'UNRATE': 1948-01-01    3.4
 1948-02-01    3.8
 1948-03-01    4.0
 1948-04-01    3.9
 1948-05-01    3.5
              ... 
 2024-10-01    4.1
 2024-11-01    4.2
 2024-12-01    4.1
 2025-01-01    4.0
 2025-02-01    4.1
 Length: 926, dtype: float64,
 'GFDEBTN': 1966-01-01      320999.0
 1966-04-01      316097.0
 1966-07-01      324748.0
 1966-10-01      329319.0
 1967-01-01      330947.0
                  ...    
 2023-10-01    34001494.0
 2024-0

In [21]:
def analyze_risks(PORTFOLIO_SYMBOLS,market_data,economic_data):
    template_risk = """Analyze market risks for a portfolio containing {symbols} given:
    Economic Indicators: {econ_data}
    1-Year Volatility: {volatility}
    
    Output JSON with:
    1. required_bond_allocation (0-1)
    2. required_gold_allocation (0-1) 
    3. sector_risk_adjustments (sector: max_allocation)
    4. risk_scores (1-5 scale)
    5. scenario_analysis (recession/rate_hike cases)

    format should be like this-

    dict : {{
            "required_bond_allocation": float (0-1),
            "required_gold_allocation": float (0-1),
            "sector_risk_adjustments": {{
                "Technology": float (0-1),
                "Healthcare": float (0-1),
                ...
            }},
            "risk_scores": {{
                "interest_rate_risk": int (1-5),
                "inflation_risk": int (1-5),
                "geopolitical_risk": int (1-5),
                "market_volatility": int (1-5)
            }},
            "scenario_analysis": {{
                "recession": {{
                    "expected_loss": float (0-1),
                    "recommended_actions": list[str]
                }},
                "rate_hike": {{
                    "expected_loss": float (0-1),
                    "recommended_actions": list[str]
                }}
            }}
        }}
    
    
    
    """


    prompt = PromptTemplate(
    template=template_risk,
    input_variables=["econ_data", "econ_data", "volatility"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    econ_data = {k: v.iloc[-1] for k, v in economic_data.items()}
    volatility = market_data.pct_change().std().mean()

    chain = prompt | llm | parser
    
    response = chain.invoke({"symbols":PORTFOLIO_SYMBOLS,"econ_data": econ_data, "volatility": volatility})
    
    # response = llm.invoke(prompt.format(
    #     symbols=PORTFOLIO_SYMBOLS,
    #     econ_data=econ_data,
    #     volatility=round(volatility, 4)
    # ))

    return {"risk_factors": response}

In [20]:
analyze_risks(PORTFOLIO_SYMBOLS,market_data,economic_data)

output from analyze_risks:
 {'required_bond_allocation': 0.25, 'required_gold_allocation': 0.2, 'sector_risk_adjustments': {'Technology': 0.35, 'Healthcare': 0.25, 'Consumer Goods': 0.2, 'Financials': 0.25, 'Industrials': 0.15}, 'risk_scores': {'interest_rate_risk': 4, 'inflation_risk': 3, 'geopolitical_risk': 2, 'market_volatility': 2}, 'scenario_analysis': {'recession': {'expected_loss': 0.15, 'recommended_actions': ['Increase bond allocation to 30%', 'Reduce exposure to high-risk sectors like Technology', 'Rebalance portfolio to include more defensive sectors']}, 'rate_hike': {'expected_loss': 0.1, 'recommended_actions': ['Shift to shorter-duration bonds to mitigate interest rate risk', 'Increase gold allocation to 25% as a hedge against inflation', 'Consider adding dividend-paying stocks for stability']}}}


{'risk_factors': {'required_bond_allocation': 0.25,
  'required_gold_allocation': 0.2,
  'sector_risk_adjustments': {'Technology': 0.35,
   'Healthcare': 0.25,
   'Consumer Goods': 0.2,
   'Financials': 0.25,
   'Industrials': 0.15},
  'risk_scores': {'interest_rate_risk': 4,
   'inflation_risk': 3,
   'geopolitical_risk': 2,
   'market_volatility': 2},
  'scenario_analysis': {'recession': {'expected_loss': 0.15,
    'recommended_actions': ['Increase bond allocation to 30%',
     'Reduce exposure to high-risk sectors like Technology',
     'Rebalance portfolio to include more defensive sectors']},
   'rate_hike': {'expected_loss': 0.1,
    'recommended_actions': ['Shift to shorter-duration bonds to mitigate interest rate risk',
     'Increase gold allocation to 25% as a hedge against inflation',
     'Consider adding dividend-paying stocks for stability']}}}}

In [24]:
def optimize_portfolio(prices):
    # prices = state["market_data"]
    returns = expected_returns.mean_historical_return(prices)
    cov_matrix = risk_models.exp_cov(prices)
    
    ef = EfficientFrontier(returns, cov_matrix)
    
    # Basic diversification constraint
    ef.add_constraint(lambda w: w <= 0.15)  # Max 15% per stock
    
    # Add bond allocation constraint
    bond_indices = [i for i, s in enumerate(PORTFOLIO_SYMBOLS) if s in BOND_SYMBOLS]
    if bond_indices:
        ef.add_constraint(lambda w: sum(w[i] for i in bond_indices) >= 
                         state["risk_factors"]["required_bond_allocation"])
    
    # Add gold allocation constraint
    if GOLD_SYMBOL in PORTFOLIO_SYMBOLS:
        gold_index = PORTFOLIO_SYMBOLS.index(GOLD_SYMBOL)
        ef.add_constraint(lambda w: w[gold_index] >= 
                         state["risk_factors"]["required_gold_allocation"])
    
    # Sector constraints from both user and risk analysis
    sector_map = [SYMBOL_SECTORS[s] for s in PORTFOLIO_SYMBOLS]
    for sector, max_alloc in {**state["constraints"]["max_sector_allocation"],
                              **state["risk_factors"]["sector_risk_adjustments"]}.items():
        sector_indices = [i for i, s in enumerate(sector_map) if s == sector]
        if sector_indices:
            ef.add_constraint(lambda w, si=sector_indices: sum(w[si]) <= max_alloc)
    
    # Optimize based on risk tolerance
    if state["constraints"]["risk_tolerance"] == 'low':
        ef.min_volatility()
    elif state["constraints"]["risk_tolerance"] == 'medium':
        ef.max_sharpe()
    else:
        target_return = returns.mean() * 1.2  # 20% higher than average
        ef.efficient_return(target_return)
    
    return {"weights": ef.clean_weights()}

In [22]:
expected_returns.mean_historical_return(market_data)

Ticker
AAPL    0.090740
AMZN    0.052120
BND     0.006063
GLD     0.172360
GOOG    0.037451
JNJ     0.005642
MA      0.154382
MSFT    0.076289
NVDA    0.601772
PG      0.058476
TLT    -0.088061
TSLA   -0.105825
V       0.167237
dtype: float64

In [25]:
cov_matrix = risk_models.exp_cov(market_data)

In [26]:
cov_matrix

Ticker,AAPL,AMZN,BND,GLD,GOOG,JNJ,MA,MSFT,NVDA,PG,TLT,TSLA,V
Ticker,,,,,,,,,,,,,
AAPL,0.062405,0.028879,0.000835,-0.000814,0.032483,0.001125,0.015533,0.027184,0.031753,0.002973,0.000335,0.069311,0.013255
AMZN,0.028879,0.086078,-0.001103,0.001345,0.058763,-0.012416,0.018649,0.046605,0.081704,-0.008017,-0.005002,0.109618,0.016988
BND,0.000835,-0.001103,0.002566,0.001385,-0.001561,0.002259,0.001054,-0.001224,-0.002748,0.002188,0.006556,-0.002985,0.001186
GLD,-0.000814,0.001345,0.001385,0.021124,0.005473,-0.000120,0.000383,0.002077,0.012065,0.001580,0.003141,0.002334,0.000285
GOOG,0.032483,0.058763,-0.001561,0.005473,0.094818,-0.013763,0.013189,0.037266,0.076748,-0.012735,-0.006512,0.116774,0.013951
JNJ,0.001125,-0.012416,0.002259,-0.000120,-0.013763,0.026552,0.006452,-0.011050,-0.045116,0.013576,0.005686,-0.023705,0.006582
MA,0.015533,0.018649,0.001054,0.000383,0.013189,0.006452,0.033273,0.009862,0.007491,0.011586,0.002612,0.039211,0.028265
MSFT,0.027184,0.046605,-0.001224,0.002077,0.037266,-0.011050,0.009862,0.052827,0.071469,-0.003501,-0.004229,0.071911,0.007370
NVDA,0.031753,0.081704,-0.002748,0.012065,0.076748,-0.045116,0.007491,0.071469,0.323567,-0.037862,-0.006590,0.156072,0.007419
